# Multiobjective Model Implementation

## Reading the instance and loding the libraries and classes

In [57]:
# implementaion of network and multicast groups in python.
from classes import Network, MulticastGroup

# implementation of the problem and an wrapper to the solver
from multicastpacking import MulticastPacking, solver

# instance reander
import reader

# loading gurobi library
from gurobipy import *

#an instance of the problem
file = "/Users/romeritocampos/HD/Doutorado/MPP_instances/n30/b30_1.brite"

# readingthe and returning its links
links = reader.get_network (file)

# creating the network object
net = Network (links, nodes = 30)

# creating the multicast groups associated with `file` instance
mgroups = [MulticastGroup (g) for g in reader.get_groups (file) ]

# creating the problem
problem = MulticastPacking (net, mgroups)

# initilizing some useful variables
KSIZE = len(problem.groups)+1
NODES = net.nodes+1

## Implementation of the Base Model

In [58]:
# creating a Gurobi Model object
m = Model ("Multiobjective model")

# variables sets
var_x = {}
var_y = {}

# defining the var_x set of variables
for link in net.links:
    for k in xrange (1, KSIZE):
        for d in problem.groups[k-1].members:
            x=link[0],link[1],k,d,
            y=link[1],link[0],k,d,
            var_x[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
            var_x[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
            
m.update ()

# defining the var_y set of variables
for k in xrange (1, KSIZE):
    for link in net.links:
        x=link[0],link[1],k
        y=link[1],link[0],k
        var_y[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
        var_y[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

m.update ()

In [59]:
# constraints of flow 1
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow1',k,d
        m.addConstr (
            quicksum ( var_x[x] for x in tuplelist (var_x).select ('*',sk,k,d) )
            -
            quicksum ( var_x[x] for x in tuplelist (var_x).select (sk,'*',k,d) )
            == -1,
            name=str(_name)
        )
m.update ()

In [60]:
# constraints of flow 2
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for j in xrange(1,NODES):
            sk = problem.groups[k-1].source
            _name='flow2',k,d,j,       
            m.addConstr (
                quicksum(
                   var_x[x] for x in tuplelist (var_x).select ('*',j,k,d) 
                    if x[1] not in [sk, d]
                )
                -
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select (j,'*',k,d) 
                    if x[0] not in [sk, d]
                )
                == 0,
                name=str(_name)
            )            

m.update ()

In [61]:
# constraints of flow 3
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow3',k,d
        m.addConstr (
            quicksum (
                var_x[x] for x in tuplelist (var_x).select ('*',d,k,d)
            )
            -
            quicksum (
                var_x[x] for x in tuplelist (var_x).select (d,'*',k,d)
            )
            == 1,
            name=str(_name)
        )
        
m.update ()

In [62]:
# constaints used to force y variables set to 1 when there is flow associated to x variable realated to edge 
# represented by y.

for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for link in net.links:
            x=link[0],link[1],k,d,
            y=link[0],link[1],k
            _name='r4',link[0],link[1],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )            
            x=link[1],link[0],k,d,
            y=link[1],link[0],k
            _name='r4',link[1],link[0],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )
m.update ()

In [63]:
# set of constaints used to avoid leaf non-terminals
for k in xrange(1,KSIZE):
    for link in net.links:
        _name='out',link[0],link[1],k
        m.addConstr(
            quicksum(
                  var_x[x] for x in tuplelist (var_x).select (link[0],link[1],k,'*')              
            )
            -
            var_y[link[0],link[1],k]
            >=
            0,
            name=str(_name)
        )
        _name='out',link[1],link[0],k,
        
        m.addConstr(
            quicksum(
                  var_x[x] for x in tuplelist (var_x).select (link[1],link[0],k,'*')              
            )
            -
            var_y[link[1],link[0],k]
            >=
            0,
            name=str(_name)
        )
m.update()

In [64]:
# constraints associated with capacitity of the network
Z = 0
for i,j in links:
		constname='cap',i,j
		m.addConstr (
			links[(i,j)][1] - 
			quicksum ( var_y[(i,j,k)] * problem.groups[k-1].traffic for k in range(1,KSIZE)) >= Z,
			name=str(constname)
		)
		constname='cap',j,i
		m.addConstr (
			links[(i,j)][1] - 
			quicksum ( var_y[(j,i,k)] * problem.groups[k-1].traffic for k in range(1,KSIZE)) >= Z,
			name=str(constname)
		)

m.update()

In [65]:
# constraints associated to the hop counting
HOP = 1
for k in xrange(1, KSIZE):
    for d in problem.groups[k-1].members:
        _name='hop',k,d
        m.addConstr(
            quicksum(
                 var_x[x] for x in tuplelist (var_x).select ('*','*',k,d)
            )
            <=
            HOP,
            name=str(_name)
        )
m.update ()

## Partial Model


$$\sum_{i \in V}{x_{ir_k}^{kd}} - \sum_{i \in V}{x_{r_{k}i}^{kd}} = -1\quad {\forall k \in K \atop \forall d \in D^k}$$

$$\sum_{(i,d) \in E}{x_{id}^{kd}} - \sum_{(d,i) \in E}{x_{di}^{kd}} = 1 \quad {\forall k \in K \atop \forall d \in D^k}$$

$$\sum_{(i,j) \in E \atop j \neq r_k,d}{x_{ij}^{kd}} - \sum_{(i,j) \in E \atop j \neq r_k,d}{x_{ji}^{kd}} = 0 \quad {\forall k \in K \atop \forall d \in D^k}$$

$$x_{ij}^{kd} \leq y_{ij}^{k} \quad \forall k \in K, d \in D^k  \text{ e }  \forall (i,j) \in E$$

$$\sum_{d \in D^k}{x_{ij}^{kd}} - y_{ij}^{k} \geq 0 \quad \forall k \in K$$

$$b_{ij}  \geq \sum_{k \in K} y_{ij}^{k} t^k \quad \forall (i,j) \in E$$

$$b_{ij}  - \sum_{k \in K} y_{ij}^{k} t^k = Z \quad \forall (i,j) \in E$$

$$ \sum_{(i,j) \in E} x_{ij}^{k} <= HOP \quad \forall k \in K, \forall d \in D^k$$

$$x_{ij}^{kd} \in \lbrace 0, 1 \rbrace , y_{ij}^{k} \in \lbrace 0, 1 \rbrace  {{\forall k \in K,  \forall d \in D^k} \atop \forall (i,j) \in E}$$



In [66]:
# adding objective function - summation of the cost of each variable included in the solution
expr  = []
for k in xrange (1, KSIZE ):
    expr.append ( quicksum (var_y[ (l[0],l[1],k) ] * links[l][0] for l in links.keys()) )
    expr.append ( quicksum (var_y[ (l[1],l[0],k) ] * links[l][0] for l in links.keys()) )

m.setObjective (quicksum (expr), GRB.MINIMIZE)
m.update ()

In [67]:
m.write ('prototype.lp')

## Running multiobjective pareto construction

In [68]:
# here goes the code to run the model multiple times


def reset_hop (model, value):
    for k in xrange(1, KSIZE):
        for d in problem.groups[k-1].members:
            _name='hop',k,d
        
            c = model.getConstrByName(str(_name))
            c.setAttr ('RHS', value)
            
    model.update ()

def reset_Z (model, value):
    for i,j in links:
        constname='cap',i,j
        c = model.getConstrByName (str(constname))
        c.setAttr ('RHS', c.rhs + value)
        constname='cap',j,i
        c = model.getConstrByName (str(constname))
        c.setAttr ('RHS', c.rhs + value)
    model.update ()




In [75]:
# this piece of code runs the model multiple times looking for all solutions
# to build the pareto for the instance used as input
m.reset ()
mm = m.copy ()
mm.params.LogToConsole = 0
mm.optimize ()
status = mm.status

if status == GRB.Status.OPTIMAL:
    print mm.objVal, 0, 0

solution_set = []
    
for y in xrange (0, 21):
    mm.reset ()
    reset_hop (mm, y)
    acc = 0
    for x in xrange (0,33):
        mm.reset ()
        mm.params.LogToConsole = 0
        acc = acc + 1
        reset_Z (mm, x)
        mm.optimize ()
        status = mm.status
        if status == GRB.Status.OPTIMAL:
            solution_set.append ((mm.objVal, x, y))
#             print (mm.objVal, x, y)       
        reset_Z (mm, -x)
    
# print ("Number of Solutions &d" % len (solution_set))
# for s in solution_set:
#     print (s)
    
# o valor máximo de capacidade residual limita o tamanho do caminho
# crescer os caminhos não aumenta Z, devido um limite na capacidade dos links (valor não unitário)
# Com isso é fácil evitar múltiplas execuções redudantes

(14284.0, 0, 5)
(14284.0, 1, 5)
(14284.0, 2, 5)
(14499.0, 3, 5)
(14499.0, 4, 5)
(14499.0, 5, 5)
(14499.0, 6, 5)
(14499.0, 7, 5)
(14499.0, 8, 5)
(14499.0, 9, 5)
(14499.0, 10, 5)
(14499.0, 11, 5)
(14499.0, 12, 5)
(14499.0, 13, 5)
(14725.0, 14, 5)
(14725.0, 15, 5)
(15043.0, 16, 5)
(15454.0, 17, 5)
(15454.0, 18, 5)
(15454.0, 19, 5)
(15454.0, 20, 5)
(15454.0, 21, 5)
(15454.0, 22, 5)
(15454.0, 23, 5)
(15630.0, 24, 5)
(15630.0, 25, 5)
(15630.0, 26, 5)
(16565.0, 27, 5)
(16720.0, 28, 5)
(17080.0, 29, 5)
(17337.0, 30, 5)
(17792.0, 31, 5)
(18467.0, 32, 5)
(13789.0, 0, 6)
(13789.0, 1, 6)
(13789.0, 2, 6)
(14004.0, 3, 6)
(14004.0, 4, 6)
(14004.0, 5, 6)
(14004.0, 6, 6)
(14004.0, 7, 6)
(14004.0, 8, 6)
(14004.0, 9, 6)
(14004.0, 10, 6)
(14004.0, 11, 6)
(14004.0, 12, 6)
(14004.0, 13, 6)
(14221.0, 14, 6)
(14221.0, 15, 6)
(14558.0, 16, 6)
(14916.0, 17, 6)
(14916.0, 18, 6)
(14916.0, 19, 6)
(14916.0, 20, 6)
(14916.0, 21, 6)
(14916.0, 22, 6)
(14916.0, 23, 6)
(14935.0, 24, 6)
(14935.0, 25, 6)
(14935.0, 26, 6)


In [73]:
m.reset ()
mm = m.copy ()

reset_hop (mm, 5)
reset_Z (mm, 32)

mm.optimize ()

status = mm.Status
if status == GRB.Status.OPTIMAL:
    print ('OK')

Optimize a model with 5616 rows, 4440 columns and 24240 nonzeros
Variable types: 0 continuous, 4440 integer (4440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 1290 rows and 1041 columns
Presolve time: 0.19s
Presolved: 4326 rows, 3399 columns, 17185 nonzeros
Variable types: 0 continuous, 3399 integer (3399 binary)

Root relaxation: objective 1.837350e+04, 1215 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 18373.5000    0   90          - 18373.5000      -     -    0s
H    0     0                    20615.000000 18373.5000  10.9%     -    0s
H    0     0                    18467.000000 18373.5000  0.51%     -    0s
     0     0 18428.0000    0   30 18467.0000 18428.0000  0.21%     -    0s

Cutting planes:
  MI

In [118]:
# filtering the pareto

print type (solution_set)
        
def non_dominate (lista, ind):
    for x in lista:
        if x[0] <= ind[0] and x[1] >= ind[1] and x[2] <= ind[2] and ind != x:
            return False
    return True

lista = []
for x in solution_set:
    if non_dominate (solution_set, x):
        lista.append (x)
print (len(lista))
print (lista)

<type 'list'>
44
[(14284.0, 2, 5), (14499.0, 13, 5), (14725.0, 15, 5), (15043.0, 16, 5), (15454.0, 23, 5), (15630.0, 26, 5), (16565.0, 27, 5), (16720.0, 28, 5), (17080.0, 29, 5), (17337.0, 30, 5), (17792.0, 31, 5), (18467.0, 32, 5), (13789.0, 2, 6), (14004.0, 13, 6), (14221.0, 15, 6), (14558.0, 16, 6), (14916.0, 23, 6), (14935.0, 26, 6), (16057.0, 27, 6), (16329.0, 28, 6), (16503.0, 29, 6), (16666.0, 30, 6), (16889.0, 31, 6), (17289.0, 32, 6), (13641.0, 2, 7), (13856.0, 13, 7), (14073.0, 15, 7), (14512.0, 16, 7), (14679.0, 23, 7), (14787.0, 26, 7), (15871.0, 27, 7), (15906.0, 28, 7), (16381.0, 29, 7), (16508.0, 30, 7), (16731.0, 31, 7), (17037.0, 32, 7), (15719.0, 28, 8), (16194.0, 29, 8), (16425.0, 30, 8), (13813.0, 13, 9), (14668.0, 17, 9), (15712.0, 28, 9), (16187.0, 29, 9), (16238.0, 30, 9)]
